i will duplicate same as in report - if I  would start over I would create 1 notebook in kaggle,
create .py files in repo and will create clean notebooks using repo, lesson learned.

everything except data, prompt - same as in confusing_samples.ipynb

I duplicate comments that occur below:
you can find list of used names in repo by NLP/datasets/partial_datasets/MT_above_5000.csv
you can find generated results in repo by datasets/partial_datasets/synthetic_batch_1.zip

In [ ]:
!pip install transformers bitsandbytes-cuda110 bitsandbytes

In [ ]:

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import pandas as pd
from tqdm import tqdm
from typing import List
import spacy
import re
import json
import gc  # Added import for garbage collection

# Load Spacy model
nlp = spacy.load("en_core_web_sm")

# Local model and tokenizer paths
model_dir = "/kaggle/input/llama-3.1/transformers/8b-instruct/2/"

# Load LLAMA model and tokenizer
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=False
)

model = AutoModelForCausalLM.from_pretrained(
    model_dir, 
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

# Adjust model configuration
model.config.use_cache = False
model.config.pretraining_tp = 1

# Load CSV file
# This dataset was generated by elevation_names_extraction.ipynb
# by setting min_elevation extraction parameter to 5000
# but I quickly understood that it's mistake and sampled randomly for next batch
# it can be found in NLP/datasets/partial_datasets/MT_above_5000.csv
mt_df = pd.read_csv("/kaggle/input/elevations-above-5000/MT_above_5000.csv")

mt_name_samples = list(mt_df["name"])


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [5]:
system_prompt = """
You are an advanced AI trained in natural language processing and synthetic data generation. Your task is to read the following mountain name and generate 10 unique sentences using mountain name.
Make main focus on diversifying sentences - sentences structures and words.
For each mountain name, provide a JSON output with the following structure:
- "actual mountain name": [ array that contains 10 sentences which are strings]

Make sure to extract the exact string of the mountain name without any corrections or conversions.
For each sentence highlight name of peak, mountain or range by setting "||" around it. Do not provide any explanations.
Only respond with the JSON structured data.

### Example 1:
Input:
'
Cerro Torre
'

Output:
[
    {
        "Cerro Torre": [
            "The imposing peak of ||Cerro Torre|| is one of the most challenging climbs in the ||Patagonia Range||.",
            "He's accustomed to climbing on the ||Cerro Torre||.",
            "Many climbers dream of scaling the icy slopes of ||Cerro Torre||, nestled within the majestic ||Patagonia Range||.",
            "Explorers often describe the air around ||Cerro Torre|| as crisp and invigorating, a testament to its pristine surroundings.",
            "The rugged terrain of ||Cerro Torre|| serves as a natural barrier, preserving the unique biodiversity of the region.",
            "Let's go for a walk to the ||Cerro Torre||, where the world seems to stretch endlessly below.",
            "The treacherous weather around ||Cerro Torre|| makes it one of the most daunting peaks in the ||Patagonia Range|| to conquer.",
            "Mountain ||Cerro Torre|| is revered by mountaineers for its challenging ascents and breathtaking vistas.",
            "The first light of dawn on ||Cerro Torre|| unveils a panorama of unparalleled beauty.",
            "You'll see mount ||Cerro Torre|| through the window."
        ]
    }
]

### Example 2:
Input:
'
Mount Foraken
'

Output:
[
    {
        "Mount Foraken": [
            "The local guides often share stories of ||Mount Foraken's|| hidden trails and secluded beauty within the ||Alaska Range||.",
            "||Mount Foraken|| is renowned for its serene ambiance and untouched surroundings in the heart of the ||Alaska Range||.",
            "The sunrise at ||Mount Foraken|| paints the sky in shades of orange and pink, leaving travelers mesmerized by the beauty of the ||Alaska Range||.",
            "The journey to ||Mount Foraken|| passes through dense forests and meadows filled with wildflowers, all nestled in the breathtaking ||Alaska Range||.",
            "Many travelers consider a visit to ||Mount Foraken||to be a spiritual and refreshing escape.",
            "||Mount Foraken||is a towering symbol of resilience for the native community.",
            "They couldn't decide whether to go to the ||Mount Foraken||or the sea.",
            "From afar, ||Mount Foraken||stands majestic, wrapped in clouds and mystery.",
            "The rugged paths leading to ||Mount Foraken||are a test of endurance for adventurers.",
            "Every season brings a unique charm to ||Mount Foraken||, from snowy peaks to lush greenery."
        ]
    }
]

Please continue with task and stop only after generating valid output for given mountain by user by outputting '### Output ends here.' don't forget this strict rule, strictly output valid JSON structure, never forget [] brackets. 
"""

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList
from torch import cuda, LongTensor, FloatTensor
import os

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

def extract_json_from_response(response):
    try:
        response = response[response.find("User:"):]
        response = response[:response.find("### Output ends here.")]
        actual_jsonlike = response[response.find("["):]
        match = re.search(r'(\[.*)', actual_jsonlike, re.DOTALL)
        if match:
            json_content = match.group(0).strip()
            if not json_content.endswith(']'):
                json_content += ']'
            return json.loads(json_content)
    except json.JSONDecodeError as e:
        print(f"Failed to decode JSON: {e}")
        return []



def create_stopping_criteria(stop_words, tokenizer, device):
    class StoppingCriteriaSub(StoppingCriteria):
        def __init__(self, stops = [], device=device, encounters = 1):
            super().__init__()
            self.stops = stops = [stop.to(device) for stop in stops]

        def __call__(self, input_ids: LongTensor, scores: FloatTensor) -> bool:
            last_token = input_ids[0][-1]
            for stop in self.stops:
                if tokenizer.decode(stop) == tokenizer.decode(last_token):
                    return True
            return False

    stop_word_ids = [tokenizer(stop_word,
                               return_tensors="pt", 
                               add_special_tokens=False)["input_ids"].squeeze() 
                               for stop_word in stop_words]

    stopping_criteria = StoppingCriteriaList([StoppingCriteriaSub(stops=stop_word_ids)])
    return stopping_criteria


stop_words_list = ["Output ends"]
stopping_criteria = None
if stop_words_list is not None:
    stopping_criteria = create_stopping_criteria(stop_words_list, tokenizer, device)

def write_batch_to_json(batch_entities, start_idx):
    filename = os.path.join("/kaggle/working/generated/", "batch_" + str(start_idx) + ".json")
    with open(filename, "w", encoding='utf-8') as f:
        json.dump(batch_entities, f)
        

def predict_entities_in_batches(test_dataset, model, tokenizer, system_prompt, batch_size=8):
    text_generation_pipeline = pipeline("text-generation", model=model, tokenizer=tokenizer)

    with torch.no_grad():
        for i in tqdm(range(0, len(test_dataset), batch_size), desc="Processing batches"):
            batch_entities = []
            if (i + batch_size) > len(test_dataset):
                batch_prompts = test_dataset[i:]
            else:
                batch_prompts = test_dataset[i:i + batch_size]

            chat_inputs = [
                f"{system_prompt}\nUser: {prompt}" for prompt in batch_prompts
            ]
            results = text_generation_pipeline(chat_inputs, max_new_tokens=400, do_sample=False, stopping_criteria=stopping_criteria)
            gc.collect()
            torch.cuda.empty_cache()
            for result in results:
                generated_text = result[0]['generated_text']
                entities = extract_json_from_response(generated_text)
                batch_entities.append(entities)
            
            write_batch_to_json(batch_entities, i) 


os.makedirs("/kaggle/working/generated/", exist_ok=True)
# generated results in repo by datasets/partial_datasets/synthetic_batch_1.zip
dataset = mt_name_samples[:600]
processed_data = predict_entities_in_batches(dataset, model, tokenizer, system_prompt)

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/generated', "zip", '/kaggle/working/generated/')